In [5]:
# pip install the required packages if running in Colab
try:
  import google.colab
  IN_COLAB = True
  %pip install summerepi
except:
  IN_COLAB = False

In [6]:
# Standard imports, plotting option and constant definition
from datetime import datetime, timedelta
from typing import List, Union
import pandas as pd
import plotly.express as px
import numpy as np

from summer import CompartmentalModel
from summer import Stratification

pd.options.plotting.backend = "plotly"

COVID_BASE_DATE = datetime(2019, 12, 31)

In [7]:
# The data import module lives in a file on AuTuMN github - download it for colab use
if IN_COLAB:
    !wget https://raw.githubusercontent.com/monash-emu/AuTuMN/master/notebooks/capacity_building/philippines/import_phl_data.py

import import_phl_data
from import_phl_data import get_population_and_epi_data

In [8]:
# Shareable google drive links
PHL_DOH_LINK = "1NgyUqoM48ZUHksLAfpdXy5dp1tc2c_wF"  # sheet 05 daily report.
PHL_FASSSTER_LINK = "1hydJK_vfTCPz-jgnK7P1_ZSyq9__dZtJ" # Fassster google drive zip file.
initial_population, df = get_population_and_epi_data(PHL_DOH_LINK, PHL_FASSSTER_LINK) 

# We define a day zero for the analysis
COVID_BASE_DATE = datetime(2019, 12, 31)

https://drive.google.com/uc?id=1NgyUqoM48ZUHksLAfpdXy5dp1tc2c_wF&export=download&confirm=t


c:\Users\rrag0004\Models\AuTuMN_new\notebooks\capacity_building\philippines\import_phl_data.py:38: DtypeWarning: Columns (5,19) have mixed types. Specify dtype option on import or set low_memory=False.
  


In [9]:
# Define the model running period and convert to a numeric representation
start_date = datetime(2021, 1, 30)
end_date = start_date + timedelta(days=300)
start_date_int = (start_date - COVID_BASE_DATE).days
end_date_int = (end_date - COVID_BASE_DATE).days

In [10]:
# Define a target set of observations to compare against our modelled outputs later
notifications_target = df[start_date: end_date]["cases"]

# Model
## Define a model

In [11]:
def build_unstratified_model(parameters: dict) -> CompartmentalModel:
    """
    Create a compartmental model, with the minimal compartmental structure needed to run and produce some sort of 
    meaningful outputs.
    
    Args:
        parameters: Flow parameters
    Returns:
        A compartmental model currently without stratification applied
    """

    model = CompartmentalModel(
        times=(parameters["start_time"], parameters["end_time"]),
        compartments=["S", "E", "I", "R"],
        infectious_compartments=["I"],
        ref_date=COVID_BASE_DATE
    )

    infectious_seed = parameters["infectious_seed"]

    model.set_initial_population(distribution={"S": initial_population - infectious_seed, "E": 0, "I": infectious_seed})
    
    # Susceptible people can get infected
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=parameters["contact_rate"], 
        source="S", 
        dest="E"
    )
    # Expose people transition to infected
    model.add_transition_flow(
        name="progression",
        fractional_rate=parameters["progression_rate"],
        source="E",
        dest="I",
    )

    # Infectious people recover
    model.add_transition_flow(
        name="recovery",
        fractional_rate=parameters["recovery_rate"],
        source="I",
        dest="R",
    )

    # Add an infection-specific death flow to the I compartment
    model.add_death_flow(name="infection_death", death_rate=parameters["death_rate"], source="I")


    # We will also request an output for the 'progression' flow
    model.request_output_for_flow("progressions", "progression")

    # Finally, we request an output for the notifications, assumed to be a fraction of the 'progression' flow
    model.request_function_output(
        name="notifications", 
        func=lambda x: x * parameters['reporting_fraction'],
        sources=["progressions"]
    )

    return model

# Strain stratification

We aim to capture two different viral strains with our model: the wild-type strain and a newly emerging variant. 

The new variant may have a different level of transmissibility and could be associated with a different risk of death, compared to the ancestral strain. We want to capture these differences with our model using a strain stratification.

In [12]:
from summer import StrainStratification
def get_strain_stratification(
    compartments_to_stratify: List[str], 
    voc_params: dict
) -> Stratification:
    """
    Create a summer stratification object that stratifies compartments into
    strata, which are intended to represent infectious disease strains.
    
    Args:
        compartments_to_stratify: List of the compartments to stratify
        voc_params: A dictionary which speicifies the infectiousness and severity of strains
    Returns:
        A summer stratification object to represent strain stratification (not yet applied)
    """
    strata = ["wild", "variant"]
    strat = StrainStratification(name="strain", strata=strata, compartments=compartments_to_stratify)

    # At the start of the simulation, a certain proportion of infected people have the variant strain.
    strat.set_population_split({'wild': 1. - voc_params["variant_start_prop"], 'variant': voc_params["variant_start_prop"]})

    # Adjusting the death risk associated with the new variant
    strat.set_flow_adjustments("infection_death", {
        "wild": None,
        "variant": voc_params["variant_rel_ifr"],
    })

    # Adjusting the transmissibility of the new variant.
    strat.set_flow_adjustments("infection", {
        "wild": None,
        "variant": voc_params["variant_rel_transmissibility"],
    })


    # Adjusting the progression rate of the new variant.
    strat.set_flow_adjustments("progression", {
        "wild": None,
        "variant": voc_params["variant_rel_progression"],
    })
    
    return strat

### Combine the two processes together

In [13]:
def build_strain_stratified_model(
    parameters: dict,
    compartments_to_stratify:List[str],
    voc_params: dict
) -> CompartmentalModel:
    """
    Get the model object with the strain stratification applied to it.
    
    Arguments:
        parameters: A dictionary containing the parameter values to use
        compartments_to_stratify: A list of compartments to which the stratification is applied
        voc_params: A dictionary which speicifies the infectiousness and lethality of the variant 
    Returns:
        The model object        
    """
    
    # Get an unstratified model object
    model = build_unstratified_model(parameters)

    # Get and apply the stratification
    strain_strat = get_strain_stratification(compartments_to_stratify, voc_params)
    model.stratify_with(strain_strat)

    # Track proporiton of new variant among incident cases
    model.request_output_for_flow("variant_progressions", "progression", dest_strata={'strain': "variant"})
    model.request_function_output(
        "prop_variant",
        func=lambda v_progress, total_progress: v_progress / total_progress,
        sources=["variant_progressions", "progressions"]
    )

    return model

### Actually run the models

In [14]:
# Build and run the stratified model with some arbitrary parameters
general_params = {
    "contact_rate": 0.2,
    "progression_rate": 0.2,
    "recovery_rate": 0.2,
    "death_rate": 0.,
    "reporting_fraction": 0.10,
    "start_time": start_date_int,
    "end_time": end_date_int,
    "infectious_seed": 100.
}

voc_params = {
    "variant_start_prop": .1,
    "variant_rel_ifr": 1.2,
    "variant_rel_transmissibility": 2.,
    "variant_rel_progression": .5
}

compartments_to_stratify = ["E", "I", "R"]
strain_strat_model = build_strain_stratified_model(general_params, compartments_to_stratify, voc_params)

In [15]:
strain_strat_model.run()

notifications = strain_strat_model.get_derived_outputs_df()["notifications"]

pd.DataFrame(
    {
        "observed": notifications_target,
        "notifications": notifications
    }
).plot()

C:\Users\rrag0004\AppData\Local\Temp\ipykernel_15216\2096425227.py:28: RuntimeWarning: invalid value encountered in true_divide
  func=lambda v_progress, total_progress: v_progress / total_progress,


### Visualise the proportion of new variant among incident cases

In [16]:
strain_strat_model.get_derived_outputs_df()["prop_variant"].plot()

# Having two strains present from the beginning may not be realistic...

We may want to have the wild-type virus only when we start the model and the new variant emerging later. To do this, we will use the existing strain stratified model with variant_start_prop = 0. However, we will need to add an extra feature to the model to simulate the new variant's emergence... Let's use an importation flow to seed infections with the new variant.

In [17]:
def build_strain_strat_model_with_seeding(
    parameters: dict,
    compartments_to_stratify:List[str],
    voc_params: dict
) -> CompartmentalModel:
    """
    Get the model object with the strain stratification applied to it and using variant infections seeding after model start time.
    
    Arguments:
        parameters: A dictionary containing the parameter values to use
        strata: The strata to be implemented in the strain stratification
        compartments_to_stratify: A list of compartments to which the stratification is applied
        voc_params: A dictionary which speicifies the infectiousness and lethality of the variant as well as its emergence time.
    Returns:
        The model object        
    """
    # Build a strain stratified model using the previousy created function
    model = build_strain_stratified_model(parameters, compartments_to_stratify, voc_params)

    # Create a time-variant infection seeding function for the new variant
    def variant_seeding_rate(time, computed_values):
        if voc_params["variant_emergence_time"] <= time <= voc_params["variant_emergence_time"] + 10.:
            return 100.
        else:
            return 0.

    # Add an importation flow to seed infections with the new variant
    model.add_importation_flow(
        name="variant_imports", 
        num_imported=variant_seeding_rate, 
        dest="I", 
        dest_strata={"strain": "variant"},
        split_imports=True  # if multiple destination compartments, should we split num_imported between them or should we replicate this number.
    )

    return model

In [18]:
# Build and run the stratified model with some arbitrary parameters
general_params = {
    "contact_rate": 0.3,
    "progression_rate": 0.4,
    "recovery_rate": 0.2,
    "death_rate": 0.,
    "reporting_fraction": 0.04,
    "start_time": start_date_int,
    "end_time": end_date_int,
    "infectious_seed": 5000.
}

voc_params = {
    "variant_start_prop": 0.,
    "variant_rel_ifr": 1.2,
    "variant_rel_transmissibility": 3.,
    "variant_rel_progression": .5,
    "variant_emergence_time": 500  # in days since 31 Dec 2019
}

compartments_to_stratify = ["E", "I", "R"]
new_strain_strat_model = build_strain_strat_model_with_seeding(general_params, compartments_to_stratify, voc_params)

In [19]:
new_strain_strat_model.run()

notifications = new_strain_strat_model.get_derived_outputs_df()["notifications"]

pd.DataFrame(
    {
        "observed": notifications_target,
        "notifications": notifications
    }
).plot()


C:\Users\rrag0004\AppData\Local\Temp\ipykernel_15216\2096425227.py:28: RuntimeWarning:

invalid value encountered in true_divide



In [20]:

new_strain_strat_model.get_derived_outputs_df()["prop_variant"].plot()

# Manual calibration to notifications

In [21]:
calibrated_general_params = {
    "contact_rate": 0.3,
    "progression_rate": 0.4,
    "recovery_rate": 0.2,
    "death_rate": 0.,
    "reporting_fraction": 0.03,
    "start_time": start_date_int,
    "end_time": end_date_int,
    "infectious_seed": 5000.
}

calibrated_voc_params = {
    "variant_start_prop": 0.,
    "variant_rel_ifr": 1.8,
    "variant_rel_transmissibility": 3,
    "variant_rel_progression": .5,
    "variant_emergence_time": 500
}

compartments_to_stratify = ["E", "I", "R"]
calibrated_model = build_strain_strat_model_with_seeding(calibrated_general_params, compartments_to_stratify, calibrated_voc_params)

In [22]:
calibrated_model.run()

notifications = calibrated_model.get_derived_outputs_df()["notifications"]

pd.DataFrame(
    {
        "observed": notifications_target,
        "notifications": notifications
    }
).plot()

C:\Users\rrag0004\AppData\Local\Temp\ipykernel_15216\2096425227.py:28: RuntimeWarning:

invalid value encountered in true_divide

